In [ ]:
# Standard imports and configuration
import logging
from pathlib import Path
import os
import sys
import time
import datetime
import json
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.gridspec import GridSpec


# Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")
logger = logging.getLogger(__name__)

# Directories (overridable via environment)
DATA_DIR = Path(os.environ.get("DATA_DIR", "."))  # default: current directory
MODELS_DIR = Path(os.environ.get("MODELS_DIR", "retrain_models"))  # default: retrain_models in current directory
OUT_DIR = Path(os.environ.get("OUT_DIR", "."))  # default: current directory

logger.info("Imports loaded. DATA_DIR=%s MODELS_DIR=%s OUT_DIR=%s", DATA_DIR, MODELS_DIR, OUT_DIR)

In [ ]:
def _load_model(name):
    path = MODELS_DIR / name
    try:
        m = keras.models.load_model(str(path))
        logger.info("Loaded model: %s", name)
        return m
    except Exception as exc:
        logger.warning("Could not load model %s: %s", name, exc)
        return None

Load the models from the best models from the hyperparameter tuning of the all the binary cases. 

In [ ]:
# Minimal robust model loading (keeps original variable names)

model_A_B = _load_model("model_A_B")
model_A_C = _load_model("model_A_C")
model_A_C1 = _load_model("model_A_C1")
model_A_C2 = _load_model("model_A_C2")
model_B_C = _load_model("model_B_C")
model_B_C1 = _load_model("model_B_C1")
model_B_C2 = _load_model("model_B_C2")
model_C_C1 = _load_model("model_C_C1")
model_C_C2 = _load_model("model_C_C2")
model_C1_C2 = _load_model("model_C1_C2")

In [ ]:
# Load input arrays (platform-independent)
interp_path = DATA_DIR / "chime_interp_frbs.npz"
model_path = DATA_DIR / "chime_model_frbs.npz"

if not interp_path.exists() or not model_path.exists():
    logger.error("Required data files missing: %s %s", interp_path.as_posix(), model_path.as_posix())
else:
    with np.load(interp_path) as arr:
        frb_data = arr["arr_0"].astype(np.float32)
    with np.load(model_path) as arr:
        frb_model_data = arr["arr_0"].astype(np.float32)
    logger.info("Loaded data shapes: frb_data=%s frb_model_data=%s", frb_data.shape, frb_model_data.shape)

In [ ]:
##### Plot one the images for sanity check  ########
test_image = frb_data[0].copy()
test_image = test_image/(test_image.max()/255)
plt.imshow(test_image)

In [ ]:
def read_to_test_data(interp_path):
    """
    Read and normalize interpolated FRB test images from DATA_DIR.
    Returns numpy array with shape (N, H, W, 1), dtype float32, values scaled 0-255.
    """

    with np.load(interp_path) as frbdata:
        frbd1 = frbdata["arr_0"]

    frbdn = []
    for img in frbd1:
        immax = img.max() 
        frbdn.append(img / (immax / 255.0))

    frbdn = np.asarray(frbdn)      
    frbdn.shape += 1,        
    logger.info("Loaded %d images, shape=%s", frbdn.shape[0], frbdn.shape)
    return frbdn

In [ ]:
test_images = read_to_test_data(str(DATA_DIR / "chime_interp_frbs.npz"))

Predictions for the all the chime bursts with 10 binary models

In [ ]:
pred_A_B   = model_A_B(test_images)
pred_A_C   = model_A_C(test_images)
pred_A_C1  = model_A_C1(test_images)
pred_A_C2  = model_A_C2(test_images)
pred_B_C   = model_B_C(test_images)
pred_B_C1  = model_B_C1(test_images)
pred_B_C2  = model_B_C2(test_images)
pred_C_C1  = model_C_C1(test_images)
pred_C_C2  = model_C_C2(test_images)
pred_C1_C2 = model_C1_C2(test_images)

In [ ]:
names = ["pred_A_B","pred_A_C","pred_A_C1","pred_A_C2",
         "pred_B_C","pred_B_C1","pred_B_C2",
         "pred_C_C1","pred_C_C2","pred_C1_C2"]
pred = [globals().get(n) for n in names if globals().get(n) is not None]

In [ ]:
def conf_matrix_from_pred(predictions, n):
    """
    Convert the 10 binary-model predictions for sample index `n` into a numeric 5x5 matrix.

    Expected `predictions` order (length 10):
      [A_vs_B, A_vs_C, A_vs_C1, A_vs_C2,
       B_vs_C, B_vs_C1, B_vs_C2,
       C_vs_C1, C_vs_C2,
       C1_vs_C2]

    Returns:
      np.ndarray shape (5,5) dtype float with np.nan on the diagonal.
      Upper-triangle entries contain the model score for (row_class vs col_class)
      and the symmetric lower-triangle is (1 - score).
    """

    def _get_score(pred_array, idx):
        arr = np.asarray(pred_array)
        if arr.ndim == 1:
            val = arr[idx]
        else:
            val = arr[idx].ravel()[0]
        return float(val)

    # mapping of prediction index -> (row, col) in 5x5 matrix (row < col)
    map_indices = [
        (0, 1), (0, 2), (0, 3), (0, 4),  # A vs B,C,C1,C2  -> preds[0..3]
        (1, 2), (1, 3), (1, 4),          # B vs C,C1,C2     -> preds[4..6]
        (2, 3), (2, 4),                  # C vs C1,C2       -> preds[7..8]
        (3, 4)                           # C1 vs C2         -> preds[9]
    ]

    mat = np.zeros((5,5) , dtype=np.float32)
    mat[mat==0]=2 

    for pred_idx, (r, c) in enumerate(map_indices):
        score = _get_score(predictions[pred_idx], n)
        score = float(score)
        mat[r, c] = round(score, 4)
        mat[c, r] = round(1.0 - score, 4)
    mat[mat==2]= np.nan    

    return mat.T

In [ ]:
thresholds = np.zeros((5,5))
thresholds[0][1] = 0.64    # A vs B 
thresholds[0][2] = 0.5    # A vs C 
thresholds[0][3] = 0.5    # A vs C1
thresholds[0][4] = 0.5    # A vs C2 
thresholds[1][0] = 0.36    # B vs A
thresholds[1][2] = 0.5    # B vs C
thresholds[1][3] = 0.2    # B vs C1
thresholds[1][4] = 0.5    # B vs C2
thresholds[2][0] = 0.5    # C vs A
thresholds[2][1] = 0.5    # C vs B
thresholds[2][3] = 0.5    # C vs C1
thresholds[2][4] = 0.5    # C s C2
thresholds[3][0] = 0.5    # C1 vs A
thresholds[3][1] = 0.8    # C1 vs B
thresholds[3][2] = 0.5    # C1 vs C
thresholds[3][4] = 0.5    # C1 vs C2
thresholds[4][0] = 0.5    # C2 vs A
thresholds[4][1] = 0.5    # C2 vs B
thresholds[4][2] = 0.5    # C2 vs C
thresholds[4][3] = 0.5    # C2 s C1


In [ ]:
max_index = []
confidences = []
n_samples = len(frb_data)

for num in range(n_samples):
    test_matrix = conf_matrix_from_pred(pred, num)
    test_matrix = np.asarray(test_matrix, dtype=np.float32) - thresholds.astype(np.float32)
    mean_rows = np.nansum(test_matrix, axis=1)

    if np.all(np.isnan(mean_rows)):
        max_index.append(np.nan)
    else:
        max_index.append(int(np.nanargmax(mean_rows)))

    confidences.append(mean_rows.astype(np.float32))

confidences = np.vstack(confidences) if len(confidences) else np.empty((0, 5), dtype=np.float32)

In [ ]:
confidences = np.array(confidences)
print(confidences)
## save he mean confidence scores which corresponds to the sum of each row
np.save('/path/to/directory/all_scores.npy', confidences)

In [ ]:
out_path = Path("All_scores.txt")

mapping = {0: "I", 1: "II", 2: "III", 3: "IV", 4: "V"}

with out_path.open("w", encoding="utf-8") as f:
    for val in max_index:
        if val is None or (isinstance(val, float) and np.isnan(val)):
            f.write("Unknown\n")
            continue
        f.write(f"{mapping.get(int(val), 'Unknown')}\n")